In [4]:
# -*- coding: utf-8 -*-
#Based on tff tutorial

In [1]:
!python --version

Python 3.7.4


In [5]:
pip install numpy==1.16.4

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.1
    Uninstalling numpy-1.18.1:
      Successfully uninstalled numpy-1.18.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf

In [24]:
import numpy as np
import collections
np.random.seed(0)
import tensorflow as tf
from tensorflow.keras.optimizers import SGD as sgd

In [5]:
!pip install tensorflow_federated

In [10]:
pip install tensorflow_federated --pre

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 389kB 940kB/s ta 0:00:01
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Could not find a version that satisfies the requirement tensorflow~=2.0.0 (from tensorflow_federated) (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0rc0, 1.1.0rc1, 1.1.0rc2, 1.1.0, 1.2.0rc0, 1.2.0rc1, 1.2.0rc2, 1.2.0, 1.2.1, 1.3.0rc0, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.4.0rc0, 1.4.0rc1, 1.4.0, 1.4.1, 1.5.0rc0, 1.5.0rc1, 1.5.0, 1.5.1, 1.6.0rc0, 1.6.0rc1, 1.6.0, 1.7.0rc0, 1.7.0rc1, 1.7.0, 1.7.1, 1.8.0rc0, 1.8.0rc1, 1.8.0, 1.9.0rc0, 1.9.0rc1, 1.9.0rc2, 1.9.0, 1.10.0rc0, 1.10.0rc1, 1.10.0, 1.10.1, 1.11.0rc0, 1.11.0rc1, 1.11.0rc2, 1.11.0, 1.12.0rc0, 1.12.0rc1, 1.12.0rc2, 1.12.0, 1.12.2, 1.12.3, 1.13.0rc0, 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 2.0

In [3]:
tf.compat.v1.enable_v2_behavior()
import tensorflow_federated as tff
from random import choices

In [7]:
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 500
NUM_CLIENTS = 3

In [9]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

97402880/97398400 [==============================] - 40s 0us/step


/home/bhrigu/.local/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [16]:
def preprocess(dataset):
  def element_fn(element):
    return collections.OrderedDict([
        ('x', tf.reshape(element['pixels'], [-1])),
        ('y', tf.reshape(element['label'], [1])),
    ])

  return dataset.repeat(NUM_EPOCHS).map(element_fn).shuffle(
      SHUFFLE_BUFFER).batch(BATCH_SIZE)

In [17]:
def make_federated_data(client_data, client_ids):
  return [preprocess(client_data.create_tf_dataset_for_client(x))
          for x in client_ids]

In [18]:
sample_clients = emnist_train.client_ids[0: NUM_CLIENTS]
federated_train_data = make_federated_data(emnist_train, sample_clients)

In [19]:
sample_clients_test = emnist_test.client_ids[0: NUM_CLIENTS]
federated_test_data = make_federated_data(emnist_test, sample_clients_test)

In [20]:
#To create federated version of our model
sample_batch = iter(federated_train_data[0]).next()
sample_batch = collections.OrderedDict([
    ('x', sample_batch['x'].numpy()),
    ('y', sample_batch['y'].numpy()),
])

In [25]:
# Create a new model
def create_compiled_keras_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(
          10, activation=tf.nn.softmax, kernel_initializer='zeros', input_shape=(784,))])

  def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
        y_true, y_pred))

  model.compile(
      loss=loss_fn,
      optimizer=sgd(learning_rate=0.02),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  return model

In [26]:
# Turn model into one that can be used with TFF
def model_fn():
  keras_model = create_compiled_keras_model()
  return tff.learning.from_compiled_keras_model(keras_model, sample_batch)


In [27]:
# Initialize training
iterative_process = tff.learning.build_federated_averaging_process(model_fn)
state = iterative_process.initialize()

In [28]:
trained_clients=[]
def get_train_data(keep_it_stupid_simple=False):
    if keep_it_stupid_simple:
        if not trained_clients:
            trained_clients.append(sample_clients)
        return federated_train_data
    sc = choices(emnist_train.client_ids, k=NUM_CLIENTS)
    for c in sc:
        while True:
            if c in trained_clients:
                sc.remove(c)
                newc=choices(emnist_train.client_ids, k=1)[0]
                if newc not in trained_clients:
                    sc.append(newc)
                    break
            else:
                break
    trained_clients.append(sc)
    new_federated_train_data = make_federated_data(emnist_train, sc)
    return new_federated_train_data

In [29]:
# Training process
for round_num in range(1, NUM_EPOCHS+1):
  federated_train_data=get_train_data(True)
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

print('Trained {:2d} clients'.format(len(trained_clients)*NUM_CLIENTS))
print(trained_clients)

round  1, metrics=<sparse_categorical_accuracy=0.12218181788921356,loss=3.1812047958374023>
round  2, metrics=<sparse_categorical_accuracy=0.1316363662481308,loss=3.169232130050659>
round  3, metrics=<sparse_categorical_accuracy=0.15054544806480408,loss=3.0420103073120117>
round  4, metrics=<sparse_categorical_accuracy=0.17599999904632568,loss=2.7166614532470703>
round  5, metrics=<sparse_categorical_accuracy=0.1679999977350235,loss=2.795884609222412>
Trained  3 clients
[['f0000_14', 'f0001_41', 'f0005_26']]


In [30]:
# Evaluation
evaluation = tff.learning.build_federated_evaluation(model_fn)

train_metrics = evaluation(state.model, federated_train_data)
print('Train metrics', str(train_metrics))

test_metrics = evaluation(state.model, federated_test_data)
print('Test metrics', str(test_metrics))

Train metrics <sparse_categorical_accuracy=0.09454545378684998,loss=3.115405559539795>
Test metrics <sparse_categorical_accuracy=0.09090909361839294,loss=3.0943548679351807>
